In [1]:
import os
import pandas as pd

### Obtendo os Dados

In [2]:
# Obtendo os produtos de interesse agropecuário:

isic = pd.read_csv('DB/NCM_ISIC.csv', encoding='latin', sep = ';')
isic_agro = isic.loc[isic['CO_ISIC_SECAO'] == 'A']

isic_agro = isic_agro[['CO_ISIC_CLASSE', 'NO_ISIC_CLASSE', 'NO_ISIC_GRUPO', 
                       'NO_ISIC_DIVISAO']]

ncm = pd.read_csv('DB/NCM.csv', encoding='latin', sep = ';')
ncm_agro = pd.merge(isic_agro, ncm, on = 'CO_ISIC_CLASSE', how = 'inner')

ncm_agro.drop(['CO_ISIC_CLASSE', 'CO_UNID', 'CO_SH6', 'CO_PPE', 'CO_PPI', 
       'CO_FAT_AGREG', 'CO_CUCI_ITEM', 'CO_CGCE_N3', 'CO_SIIT', 'CO_EXP_SUBSET', 
       'NO_NCM_ESP', 'NO_NCM_ING'], axis = 1, inplace=True)

ncm_agro.head(2)

,NO_ISIC_CLASSE,NO_ISIC_GRUPO,NO_ISIC_DIVISAO,CO_NCM,NO_NCM_POR
0,Cultivo de outras culturas perenes,Cultivo de culturas perenes,"Produção vegetal, animal e caça",14011000,Bambus para cestaria ou espartaria
1,Cultivo de outras culturas perenes,Cultivo de culturas perenes,"Produção vegetal, animal e caça",14012000,Rotins para cestaria ou espartaria


In [6]:
# Filtrando os produtos agropecuários exportados

exp = pd.DataFrame()

for file in os.listdir('DB/EXP/'):
    data = pd.read_csv('DB/EXP/'+ file, encoding='latin', sep=';')
    data = data.loc[data['CO_NCM'].isin(ncm_agro['CO_NCM'])]
    exp = pd.concat([exp, data], ignore_index=True)
exp_agro = pd.merge(exp, ncm_agro, on = 'CO_NCM', how = 'inner')
exp_agro['Operação'] = 'Exportação'

In [7]:
# Filtrando os produtos agropecuários importados

imp = pd.DataFrame()

for file in os.listdir('DB/IMP/'):
    data = pd.read_csv('DB/IMP/'+ file, encoding='latin', sep=';')
    data = data.loc[data['CO_NCM'].isin(ncm_agro['CO_NCM'])]
    imp = pd.concat([imp, data], ignore_index=True)
imp_agro = pd.merge(imp, ncm_agro, on = 'CO_NCM', how = 'inner')
imp_agro['Operação'] = 'Importação'

### Limpando os Dados

In [9]:
# Agregando os valores do frete e do seguro no preço da mercadoria

imp_agro['VL_FOB'] = imp_agro['VL_FOB'] + imp_agro['VL_FRETE'] + imp_agro['VL_SEGURO']

In [10]:
# Criando uma tabela única para as importações e exportações

comex_agro = pd.concat([imp_agro, exp_agro], ignore_index=True)

In [29]:
comex_agro.head(2)

,Peso Líquido (Kg),Custo FOB (U$),Classe,Grupo,Divisão,Nomeclatura NCM,Operação,País,Modal de Transporte,UF,Data
0,10500,86734,Cultivo de outras frutas e nozes,Cultivo de culturas perenes,"Produção vegetal, animal e caça","Nozes frescas ou secas, sem casca",Importação,Chile,MARITIMA,Santa Catarina,2021-11-01
1,1186,12544,Cultivo de outras frutas e nozes,Cultivo de culturas perenes,"Produção vegetal, animal e caça","Nozes frescas ou secas, sem casca",Importação,Chile,RODOVIARIA,Pernambuco,2021-03-01


In [13]:
# Ajustando a coluna dos países

pais = pd.read_csv('DB/PAIS.csv', encoding='latin', sep= ';')

comex_agro = pd.merge(comex_agro, pais[['CO_PAIS', 'NO_PAIS']], on='CO_PAIS', how = 'left')
comex_agro.drop('CO_PAIS', axis=1, inplace=True)
comex_agro.rename(columns = {'NO_PAIS':'País'}, inplace=True)

In [17]:
# Ajustando a coluna dos modais de transporte

via = pd.read_csv('DB/VIA.csv', encoding = 'latin', sep = ';')

comex_agro = pd.merge(comex_agro, via[['CO_VIA', 'NO_VIA']], on='CO_VIA', how = 'left')
comex_agro.drop('CO_VIA', axis=1, inplace=True)
comex_agro.rename(columns = {'NO_VIA':'Modal de Transporte'}, inplace=True)


In [19]:
# Ajustando a coluna dos estados

uf = pd.read_csv('DB/UF.csv', encoding= 'latin', sep=';')

comex_agro = pd.merge(comex_agro, uf[['SG_UF', 'NO_UF']], 
                    left_on='SG_UF_NCM', right_on='SG_UF', how = 'left')
comex_agro.drop(['SG_UF_NCM', 'SG_UF'], axis=1, inplace=True)
comex_agro.rename(columns = {'NO_UF':'UF'}, inplace=True)

In [23]:
# Ajustando a coluna da data

comex_agro['Data'] = pd.to_datetime(comex_agro['CO_ANO'].astype(str) + '-' + comex_agro['CO_MES'].astype(str), format='%Y-%m')
comex_agro.drop(['CO_ANO','CO_MES'], axis = 1, inplace=True)

In [25]:
# Removendo colunas não utilizadas

comex_agro.drop(['CO_NCM', 'CO_UNID', 'CO_URF', 'QT_ESTAT', 'VL_FRETE',
                 'VL_SEGURO'], axis = 1, inplace=True)

In [28]:
# Renomeando as demais colunas

comex_agro.rename(columns = {'KG_LIQUIDO':'Peso Líquido (Kg)', 'VL_FOB':'Custo FOB (U$)',
        'NO_ISIC_CLASSE':'Classe', 'NO_ISIC_GRUPO':'Grupo',
        'NO_ISIC_DIVISAO':'Divisão', 'NO_NCM_POR':'Nomeclatura NCM'}, inplace=True)

In [30]:
# Exportando os dados
comex_agro.to_csv('DB/comex_agro.csv')